In [ ]:
!python -V

Python 3.11.7


In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import pandas as pd

#### Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16
17
18
19

Download source data 

In [2]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

In [3]:
# Read data from jan dataset and find numbers of column
import pandas as pd
df = pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

#### Answer: 18

#### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59
42.59
52.59
62.59

In [6]:

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration = df.duration.round(2)

In [7]:
df['duration'].std()

42.59435317924198

#### Ans : 42.59

#### Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%
92%
95%
98%

In [8]:
# duration to be between 1 min to 60 min  i.e 60 sec - 3600 sec
df['duration'].describe()



count    3.066766e+06
mean     1.566899e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.120000e+00
50%      1.152000e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [9]:
cleandf = df[(df['duration'] >= 1) & (df['duration'] <= 60)].reset_index(drop=True)

In [10]:
cleandf.duration.describe()

count    3.009173e+06
mean     1.420486e+01
std      9.939385e+00
min      1.000000e+00
25%      7.220000e+00
50%      1.155000e+01
75%      1.818000e+01
max      6.000000e+01
Name: duration, dtype: float64

In [13]:
print(f"Percentage of data points remaining after dropping outlier records: {3009172/3066765 * 100 }")

Percentage of data points remaining after dropping outlier records: 98.12202760889733


#### Answer : 98%

#### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them) Fit a dictionary vectorizer Get a feature matrix from it What's the dimensionality of this matrix (number of columns)?

In [15]:
cleandf.VendorID = cleandf.VendorID.astype(str)
cleandf.RatecodeID = cleandf.RatecodeID.astype(str)
cleandf.PULocationID = cleandf.PULocationID.astype(str)
cleandf.DOLocationID = cleandf.DOLocationID.astype(str)
location_col = ['PULocationID', 'DOLocationID']
train_dicts = cleandf[location_col].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

#### Answer : 515

#### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64

In [16]:
target = 'duration'
y_train = cleandf[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649261603845807

#### Answer : 7.64

#### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81


In [21]:
# Read data from jan dataset and find numbers of column

def train_pred(data, input_col = ['PULocationID', 'DOLocationID'] , target_col = 'duration'):
    val_df = pd.read_parquet(data)
    val_df['duration'] = val_df.tpep_dropoff_datetime - val_df.tpep_pickup_datetime
    val_df.duration = val_df.duration.apply(lambda td: td.total_seconds() / 60)
    val_df.duration = val_df.duration.round(2)
    df_val_clean = val_df[(val_df['duration'] >= 1) & (val_df['duration'] <= 60)].reset_index(drop=True)
    val_df = df_val_clean 
    val_df.PULocationID = val_df.PULocationID.astype(str)
    val_df.DOLocationID = val_df.DOLocationID.astype(str)
    location_col = ['PULocationID', 'DOLocationID']
    val_dicts = val_df[input_col].to_dict(orient='records')
    dv = DictVectorizer()
    X_val = dv.fit_transform(val_dicts)
    y_val = val_df[target_col].values
    lr = LinearRegression()
    lr.fit(X_val, y_val)

    y_pred_val = lr.predict(X_val)
    return mean_squared_error(y_val, y_pred_val, squared=False)

In [22]:
mse = train_pred("yellow_tripdata_2023-02.parquet", input_col = ['PULocationID', 'DOLocationID'], target_col = 'duration')
mse

7.778951036153182

#### Answer: 7.81